In [33]:
import re

In [34]:
PATH_TO_POS = "../text/twitter-datasets/train_pos_full.txt"
PATH_TO_NEG = "../text/twitter-datasets/train_neg_full.txt"
PATH_TO_TEST = "../text/twitter-datasets/test_data.txt"
PATH_TO_SUB = "./submission.csv"

# Obtaining the data from the files
## 1. For training

In [35]:
train_x = []
train_y = []

with open(PATH_TO_POS, "r") as file:
    content = file.read()
    content_lines = content.split("\n")
    train_x += content_lines[:]
    train_y += [1] * len(content_lines)

with open(PATH_TO_NEG, "r") as file:
    content = file.read()
    content_lines = content.split("\n")
    train_x += content_lines[:]
    train_y += [-1] * len(content_lines) 

## 2. For generating the predictions

In [48]:
test_data = []
test_ids = []
test_preds = []

with open(PATH_TO_TEST) as file:
    content = file.read()
    content_lines = content.split("\n")
# Read, store in test_ids and remove id from test_data
for i in range(len(content_lines)):
    if "," in content_lines[i]:
        entrySplitted = re.split(",", content_lines[i], 1)
        test_ids.append(entrySplitted[0])
        test_data.append(entrySplitted[1])

# Constructing the table with likelihood for each word to be positive or negative

In [37]:
words = {}
for x,y in zip(train_x, train_y):
    list_words = x.split(" ")
    for w in list_words:
        if w not in words:
            words[w] = [3, 1, 1]
            if y == 1:
                words[w][1] = 1
            else:
                words[w][2] = 1
        else:
            words[w][0] += 1
            if y == 1:
                words[w][1] += 1
            else:
                words[w][2] += 1

# Basic classifier for a sentence

In [45]:
def classifier(sentence):
    probaPos = 1
    probaNeg = 1
    sentenceSplitted = sentence.split(" ")
    for word in sentenceSplitted:
        if word in words:
            probaPos *= words[word][1]/words[word][0]
            probaNeg *= words[word][2]/words[word][0]
        else:
            probaPos *= 0.5
            probaNeg *= 0.5
    if probaPos / probaNeg >= 1:
        return 1
    return -1

# Do the predictions for the test data
And write them in the submission file

In [49]:
# Predictions
for x in test_data:
    test_preds.append(classifier(x))

# Writing
with open(PATH_TO_SUB, "w") as file:
    file.write("Id,Prediction\n")
    for id,pred in zip(test_ids, test_preds):
        file.write("{},{}\n".format(id, pred))